## imports

In [1]:
import architectures as ar
import data as dt
import torch

In [10]:
args = []
dilations = [2,4,8,8,4]
strides=[5,4,3,2,2]
kernels=[10,8,4,4,4]
encode_dim=32


In [11]:
vicreg = ar.VicRegWaves(args,dilations,strides,kernels,encode_dim,[32,64,128,64,32])

In [12]:
a1 = torch.normal(torch.sin(torch.linspace(0,2*torch.pi,441)),torch.tensor([1]))
a1 = a1[None,None,:]

In [13]:
senc = vicreg.encoder.strided_encoder
denc = vicreg.encoder.dilated_encoder
proj = vicreg.projector

In [18]:
d1 = denc(a1)
s1 = senc(a1)
e1 = vicreg.encoder(a1)
p1 = proj(e1)

ValueError: Expected more than 1 value per channel when training, got input size torch.Size([1, 64])

In [17]:
print(d1.shape,s1.shape,e1.shape,p1.shape)

torch.Size([1, 16, 1]) torch.Size([1, 16, 1]) torch.Size([1, 32])
